# 211OC PREDICTIVE MODELLING

### Connecting to Snowflake

In [1]:
## import the libraries
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import snowflake.connector
from sklearn.preprocessing import LabelEncoder

#### Takes Snowflake Credentials and Database Name

In [38]:
f = open("SF_Account_Info.txt", "r")

usr = f.readline()
usr = usr.replace('\n', '')
psswrd = f.readline()
psswrd = psswrd.replace('\n', '')
accnt = f.readline()
accnt = accnt.replace('\n', '')
dbName = f.readline()
dbName = dbName.replace('\n', '')

print(usr,'\n', accnt,'\n', dbName)

MCHARIPAR5 
 ob28888.east-us-2.azure 
 adbnamehere


In [84]:
# Connect to SF
conn = snowflake.connector.connect(
    user=usr,
    password=psswrd,
    account=accnt,
    role='ACCOUNTADMIN',
    warehouse='COMPUTE_WH',
    database= dbName,
    #table='FORMATTED_REPORTS',
    #schema='PUBLIC'
                                    )
print("Connected to Snowflake using " + usr + " " + accnt)

Connected to Snowflake using MCHARIPAR5 ob28888.east-us-2.azure


In [4]:
# Creates dataframe frome Snowflake SQL Table
curr = conn.cursor()
sql = 'SELECT * FROM TOTAL_SATURNCLOUD'
curr.execute(sql)

df = curr.fetch_pandas_all()

### Renaming and dropping columns

In [5]:
df.rename(columns={'TaxonomyCode':'Level5Code',
'Demographics___Gender__Person_in_Need_':'Gender',
'Demographics___Prior_or_Current_U_S__Military_Service__Person_in_Needs_Household_':'Militants'},inplace=True)

df['Level2Code'] = df['Level5Code'].str.extract('(R.)', expand=True)

df = df.drop(columns  = ['Level5Code','TaxonomyName'])


### Encoding Militants Column

In [6]:
le = LabelEncoder()

## Militants - Categorical to Numerical
taxonomy_encoded = le.fit_transform(df['Militants'])
df['Militants'] = taxonomy_encoded

df3 = pd.get_dummies(df,columns=['Level2Code','Gender'])

df3 = df3.groupby(df3['DateOfCall'], as_index=False, sort=True).sum()

### Basic number of unqiue values assessment

In [7]:
genderList = df['Gender'].unique()
for i in genderList:
    print("Number of " + i + " :"+ str(len(df[df['Gender'] == i])))

Number of Woman :26274
Number of Man :18309
Number of Other/Unknown/Cannot determine :2897


In [8]:
MilitantList = df['Militants'].unique()
for i in MilitantList:
    print("Number " + str(i) + " :" + str(len(df[df['Militants'] == i])))

Number 0 :45799
Number 1 :1681


In [9]:
df3.shape

(2732, 12)

In [ ]:
# Describe all the data
# df3.describe(include='all')

In [ ]:
# summarize missing by column
# df3.isna().sum()

In [12]:
## Separate dates for future plotting
train_dates = pd.to_datetime(df3['DateOfCall'])

## Variables for training
df4 = df3.iloc[:, 1:11].astype(float)
#df4.dtypes

NumberOfCalls                            float64
Militants                                float64
Level2Code_RD                            float64
Level2Code_RF                            float64
Level2Code_RM                            float64
Level2Code_RP                            float64
Level2Code_RR                            float64
Level2Code_RX                            float64
Gender_Man                               float64
Gender_Other/Unknown/Cannot determine    float64
dtype: object

In [13]:
## Normalize the dataset
scaler = StandardScaler()
scaler = scaler.fit(df4)
df4_scaled = scaler.transform(df4)

In [14]:
## Setting up a training/validation/testing dataset
X_train = []
y_train = []

n_future = 1 # Number of days we want to predict into the future
n_past = 14 # Number of past days we want to use to predict the future

for i in range (n_past, len(df4_scaled) - n_future +1):
    X_train.append(df4_scaled[i - n_past:i, 0:df4.shape[1]])
    y_train.append(df4_scaled[i + n_future - 1:i + n_future, 0])

## Convert to numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)

## Build a LSTM complex model with dropout, add L1 regularization, smaller weight. 

In [15]:
from tensorflow.keras.regularizers import l1
l1_penalty = 0.001

# build a two layer neural network with regularization
def build_model5():
    model = Sequential()
    model.add(LSTM(64,
                   activation = 'relu',
                   kernel_regularizer=l1(l1_penalty),
                   input_shape = (X_train.shape[1], X_train.shape[2]),
                   return_sequences = True))
    model.add(LSTM(64,
                   activation = 'relu',
                   kernel_regularizer=l1(l1_penalty),
                   return_sequences = True))
    model.add(Dropout(0.5))
    model.add(LSTM(64,
                   activation = 'relu',
                   kernel_regularizer=l1(l1_penalty),
                   return_sequences = True))
    model.add(Dropout(0.5))
    model.add(LSTM(64,
                   activation = 'relu',
                   kernel_regularizer=l1(l1_penalty),
                   return_sequences = False))
    model.add(Dropout(0.5))
    model.add(Dense(y_train.shape[1]))
    
    model.compile(optimizer = 'adam',
                  loss = 'mse',
                  metrics = ['mae'])
    return model

# fit this model/architecture to my data
history5 = build_model5()
history5.fit(X_train,
          y_train,
          epochs = 500,
          batch_size = 32,
          validation_split = 0.2,
          verbose = 1)

Epoch 1/500
68/68 [==============================] - 41s 63ms/step - loss: 3.8121 - mae: 0.8211 - val_loss: 1.9289 - val_mae: 0.7888
Epoch 2/500
68/68 [==============================] - 2s 25ms/step - loss: 1.7288 - mae: 0.8026 - val_loss: 0.9444 - val_mae: 0.6989
Epoch 3/500
68/68 [==============================] - 2s 23ms/step - loss: 1.0451 - mae: 0.7457 - val_loss: 0.7290 - val_mae: 0.6021
Epoch 4/500
68/68 [==============================] - 2s 24ms/step - loss: 0.9453 - mae: 0.7109 - val_loss: 0.6808 - val_mae: 0.5869
Epoch 5/500
68/68 [==============================] - 2s 22ms/step - loss: 0.8921 - mae: 0.6925 - val_loss: 0.6678 - val_mae: 0.5890
Epoch 6/500
68/68 [==============================] - 2s 22ms/step - loss: 0.8713 - mae: 0.6920 - val_loss: 0.6134 - val_mae: 0.5567
Epoch 7/500
68/68 [==============================] - 2s 24ms/step - loss: 0.8808 - mae: 0.6973 - val_loss: 0.6087 - val_mae: 0.5456
Epoch 8/500
68/68 [==============================] - 2s 24ms/step - loss: 0

In [16]:
#summarize model
history5.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 14, 64)            19200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 14, 64)            33024     
_________________________________________________________________
dropout (Dropout)            (None, 14, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 14, 64)            33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0

In [26]:
# plot the loss function per epoch
# plt.plot(history5.history.history['loss'],
#          color='red')
# plt.plot(history5.history.history['val_loss'],
#          color='blue')

In [27]:
# plot the loss function per epoch
# plt.plot(history5.history.history['mae'],
#          color='red')
# plt.plot(history5.history.history['val_mae'],
#          color='blue')

In [19]:
print('The min validation loss of',
      np.min(history5.history.history['val_loss']),
      ',\n was at epoch',
      np.argmin(history5.history.history['val_loss']))

print('The min validation mae of',
      np.min(history5.history.history['val_mae']),
      ',\n was at epoch',
      np.argmin(history5.history.history['val_mae']))

The min validation loss of 0.5359986424446106 ,
 was at epoch 82
The min validation mae of 0.5263829231262207 ,
 was at epoch 82


## Forecasting Future Number Of Calls

In [70]:
# Begin with the last day in training and forcast future
future_ndays = 30
forecast_period_dates = pd.date_range(list(train_dates)[-1], 
                                      periods = future_ndays, 
                                      freq = '1d').tolist()

## forecast
forecast = history5.predict(X_train[-future_ndays:])

## Perform inverse transformation to rescale back to original range
forecast_copies = np.repeat(forecast, 
                            df4.shape[1], 
                            axis = -1)
y_pred_future = scaler.inverse_transform(forecast_copies)[:, 0]

## Convert forecast timestamp to date
forecast_dates = []
for time_i in forecast_period_dates:
    forecast_dates.append(time_i.date())

df_forecast = pd.DataFrame({'DateOfCall': np.array(forecast_dates), 
                            'Forecast NumberOfCalls':y_pred_future})
df_forecast['DateOfCall'] = pd.to_datetime(df_forecast['DateOfCall'])

#df_forecast

In [71]:
Original = df3[['DateOfCall', 'NumberOfCalls']]
Original['DateOfCall'] = pd.to_datetime(Original['DateOfCall'])
Original = Original.loc[Original['DateOfCall'] >= '2021-01-01']

## Plot Observed and forecast
# sns.lineplot(Original['DateOfCall'], Original['NumberOfCalls'])
# sns.lineplot(df_forecast['DateOfCall'], df_forecast['Forecast NumberOfCalls'])

<ipython-input-71-0d25eb745860>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Original['DateOfCall'] = pd.to_datetime(Original['DateOfCall'])


### Upload Predictions to Table In Same Snowflake Database

In [72]:
# Ensures Column Names are in the Correct Format for Snowflake
def moreFormat(theInput):
        
        anthrList = []
        stringList = []
        o = 0
        
        for i in theInput:
            if i.startswith(' '):
                theInput[o] = i.strip()
            o = o + 1
            
            if len(i) > 255:
                i = i[:255]
        
            splChar = [
            '(',')','[',']','/','\\','-','.',':','\t','<','>','"','?',
            '!','`',"'",'&','^',' ','{','}','+','=','~','|',';','%','#']
            for x in splChar:
                i = i.replace(x, '_')
            anthrList.append(i)
            stringList.append(i)

        stringList = str(stringList)
        stringList = stringList.replace("'", '"')
        stringList = stringList.replace(",", ' varchar,\n')
        stringList = stringList.strip("[]")
        stringList = stringList + " varchar"

        return anthrList, stringList

In [90]:
# df_forecast.convert_dtypes()
thisVar1 = df_forecast.columns.tolist()
df_forecast.columns, stringList1 = moreFormat(thisVar1)

# Snowflake not accepting pandas datetime datatype correctly
df_forecast['DateOfCall'] = df_forecast['DateOfCall'].astype('string')

In [91]:
df_forecast.dtypes

DateOfCall                 string
Forecast_NumberOfCalls    float32
dtype: object

In [92]:
# SQL for Table Column Names and Datatypes
stringList1 = '"DateOfCall" VARCHAR, "Forecast_NumberOfCalls" FLOAT'

In [94]:
from snowflake.connector.pandas_tools import write_pandas

conn.cursor().execute("USE DATABASE {}".format(dbName.upper()))
            
# Create Snowflake Table and Columns & Upload the DF to SF
conn.cursor().execute(
    "CREATE OR REPLACE TABLE "
    "TOTAL_SATURNCLOUD_PREDICTIONS({})".format(stringList1))
success, nchunks, nrows, _ = write_pandas(conn, df_forecast, 'TOTAL_SATURNCLOUD_PREDICTIONS')

#### Changes VARCHAR to DATE

In [95]:
conn.cursor().execute('ALTER TABLE "TOTAL_SATURNCLOUD_PREDICTIONS" ADD "DATE" DATE;')


conn.cursor().execute(
    'CREATE OR REPLACE TABLE \"TOTAL_SATURNCLOUD_PREDICTIONS\" AS(SELECT to_date(\"DateOfCall\") date,\"Forecast_NumberOfCalls\"FROM \"TOTAL_SATURNCLOUD_PREDICTIONS\");')

conn.cursor().execute('alter table \"TOTAL_SATURNCLOUD_PREDICTIONS\" rename column \"DATE\" to \"DateOfCall\";')

#### Takes Shared Account Info to be Exectued in Snowflake SQL Statement

In [77]:
f1 = open("SHARED_SF_Account_Info.txt", "r")

shareName = f1.readline()
shareName = shareName.replace('\n', '')

sharedAccnt1 = f1.readline()
sharedAccnt1 = sharedAccnt1.replace('\n', '')
sharedAccnt2 = f1.readline()
sharedAccnt2 = sharedAccnt2.replace('\n', '')
sharedAccnt3 = f1.readline()
sharedAccnt3 = sharedAccnt3.replace('\n', '')
sharedAccnt4 = f1.readline()
sharedAccnt4 = sharedAccnt4.replace('\n', '')
sharedAccnt5 = f1.readline()
sharedAccnt5 = sharedAccnt5.replace('\n', '')
sharedAccnt6 = f1.readline()
sharedAccnt6 = sharedAccnt6.replace('\n', '')
sharedAccnt7 = f1.readline()
sharedAccnt7 = sharedAccnt7.replace('\n', '')
sharedAccnt8 = f1.readline()
sharedAccnt8 = sharedAccnt8.replace('\n', '')
sharedAccnt9 = f1.readline()
sharedAccnt9 = sharedAccnt9.replace('\n', '')
sharedAccnt10 = f1.readline()
sharedAccnt10 = sharedAccnt10.replace('\n', '')
sharedAccnt11 = f1.readline()
sharedAccnt11 = sharedAccnt11.replace('\n', '')
sharedAccnt12 = f1.readline()
sharedAccnt12 = sharedAccnt12.replace('\n', '')
sharedAccnt13 = f1.readline()
sharedAccnt13 = sharedAccnt13.replace('\n', '')
sharedAccnt14 = f1.readline()
sharedAccnt14 = sharedAccnt14.replace('\n', '')
sharedAccnt15 = f1.readline()
sharedAccnt15 = sharedAccnt15.replace('\n', '')

print('The Following Accounts Will Be Granted Share Access:\n' + sharedAccnt1,
      sharedAccnt2, sharedAccnt3, sharedAccnt4, sharedAccnt5, sharedAccnt6, 
      sharedAccnt7, sharedAccnt8, sharedAccnt9, sharedAccnt10, sharedAccnt11, 
      sharedAccnt12, sharedAccnt13, sharedAccnt14, sharedAccnt15)

The Following Accounts Will Be Granted Share Access:
LJ89397 OB28888 AY81562 BE88786 CU70746 EH46115 GO95187 GU57040 JR29809 SA48519 TV80860 XL66108 ZT30250 ZV91959 


In [78]:
accountList = [sharedAccnt1, sharedAccnt2, sharedAccnt3, sharedAccnt4, 
               sharedAccnt5, sharedAccnt6, sharedAccnt7, sharedAccnt8, 
               sharedAccnt9, sharedAccnt10, sharedAccnt11, sharedAccnt12,
               sharedAccnt13, sharedAccnt14, sharedAccnt15]
ACCOUNT_LIST = []
for i in accountList:
    if len(i) != 0:
        ACCOUNT_LIST.append(i)

In [79]:
aSQLstatement = 'USE database {};'.format(dbName)
sqlScript = 'create share {};'.format(shareName)
sqlScript1 = 'grant usage on database {} to share {};'.format(dbName, shareName)
sqlScript2 = 'grant usage on schema {}.public to share {};'.format(dbName, shareName)
sqlScript3 = 'grant select on all tables in schema {}.public to share {};'.format(dbName, shareName)

In [80]:
try:
    conn.cursor().execute(aSQLstatement)
    conn.cursor().execute(sqlScript)
    print('Share Created!')
except:
    print("This share already esixts in your Snowflake account.")

This share already esixts in your Snowflake account.


In [81]:
try:
    conn.cursor().execute(aSQLstatement)
    conn.cursor().execute(sqlScript1)
    conn.cursor().execute(sqlScript2)
    conn.cursor().execute(sqlScript3)
except:
    print("There was an error in the grant usage on... SQL.")

In [82]:
try:
    for x in ACCOUNT_LIST:
        sqlScript4 = 'alter share {} add accounts={};'.format(shareName, x)
        conn.cursor().execute(aSQLstatement)
        try:
            conn.cursor().execute(sqlScript4)
        except:
            print('Account {} Could Not be Added.'.format(x))
    print("Accounts Successfully Added to {}!".format(shareName))
except:
    print("There was an error in the alter share {} add accounts=... SQL.".format(shareName))

Account OB28888 Could Not be Added.
Accounts Successfully Added to thissharehere!


In [83]:
conn.cursor().close()
print("Connection to Snowflake has been closed.")

Connection to Snowflake has been closed.
